In [1]:
!git clone https://github.com/huggingface/transformers.git
!pip install transformers/.
!pip install -r transformers/examples/pytorch/language-modeling/requirements.txt
!pip install -r transformers/examples/pytorch/text-generation/requirements.txt
!pip install datasets evaluate sacrebleu
!pip install git+https://github.com/huggingface/accelerate

Cloning into 'transformers'...
remote: Enumerating objects: 115281, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 115281 (delta 172), reused 191 (delta 84), pack-reused 114927
Receiving objects: 100% (115281/115281), 109.19 MiB | 30.89 MiB/s, done.
Resolving deltas: 100% (85741/85741), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metad

In [2]:
!git clone https://github.com/sirmammingtonham/trading-fours.git

Cloning into 'trading-fours'...
remote: Enumerating objects: 970, done.
remote: Counting objects: 100% (970/970), done.
remote: Compressing objects: 100% (964/964), done.
remote: Total 970 (delta 11), reused 953 (delta 5), pack-reused 0
Receiving objects: 100% (970/970), 20.65 MiB | 25.08 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
import torch
import datasets
import evaluate
import numpy as np
import pandas as pd
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader

In [68]:
model = AutoModelForCausalLM.from_pretrained('yeeb/distilgpt2_trading-fours')
tokenizer = AutoTokenizer.from_pretrained('yeeb/distilgpt2_trading-fours')

In [80]:
test_df = pd.read_csv('/content/trading-fours/data/test.csv')
test_df = test_df[test_df['text'].str.split().apply(len) >= 90]
test_df['ref'] = test_df['text'].apply(lambda ex: ' '.join(ex.split()[100:]))
test_df['prompt'] = test_df['text'].apply(lambda ex: ' '.join(ex.split()[:100]))
# test_ds = load_dataset('csv', data_files={'test': '/content/trading-fours/data/test.csv'})

In [81]:
test_ds = datasets.Dataset.from_pandas(test_df)

In [71]:
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [82]:
tokenized_ds = test_ds.map(
      lambda ex: tokenizer(ex['prompt'], truncation=True, max_length=200),
      batched=True,
      remove_columns=['name', 'text']
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [73]:
# model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [83]:
tokenized_ds.set_format("torch")

In [84]:
tokenized_ds[0]['input_ids'].shape

torch.Size([200])

In [85]:
test_dataloader = DataLoader(
    tokenized_ds, shuffle=True, batch_size=32
)

In [86]:
metric = evaluate.load('sacrebleu')

In [89]:
def evaluate_function(model, test_dataloader, outputs):
  accelerator = Accelerator()

  model, test_dataloader = accelerator.prepare(
        model, test_dataloader
    )
  model.eval()
  for step, batch in enumerate(test_dataloader):
    print(step)
    with torch.no_grad():
      generated_tokens = model.generate(
          batch["input_ids"],
          attention_mask=batch["attention_mask"],
          max_length=tokenizer.model_max_length
      )
      refs = batch["ref"]

      generated_tokens = generated_tokens.cpu().numpy()
      # refs = refs.cpu().numpy()
      decoded_preds = tokenizer.batch_decode(generated_tokens)
      decoded_preds = [text[200:] for i, text in enumerate(decoded_preds)]
      # decoded_refs = tokenizer.decode(refs, skip_special_tokens=True)

      metric.add_batch(predictions=decoded_preds, references=refs)
      outputs.append({'prompt': batch['prompt'], 'generated': decoded_preds})
  eval_metric = metric.compute()
  print({"bleu": eval_metric["score"]})


In [90]:
from accelerate import notebook_launcher

outputs = []
notebook_launcher(evaluate_function, (model,test_dataloader,outputs))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Launching training on one GPU.
0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


2


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


3
{'bleu': 10.308629712860522}


In [91]:
outputs[0]['prompt'][0]

'A#1 G#1 A1 A0 A1 D2 F#2 G2 D3 G2 A#1 A#0 A#1 D#2 C2 F2 A1 A#1 A#0 D#2 D#1 G#2 D2 C#2 F#2 F#1 G2 D2 C#2 C#1 C2 C2 E2 F2 A#1 A#0 A1 D#2 D2 G#2 G2 D2 A#1 A#0 D#2 D#1 C2 F2 A#1 D#2 G#2 C#2 F#2 G3 D3 C#2 G2 D#2 G2 G#2 D#2 A#2 C3 D#2 D#3 D#2 G2 G2 C2 C2 G2 F2 F1 D#2 D2 C2 B1 A#2 A#1 D2 F2 A#1 F2 D#2 A#2 D#2 D#2 G2 G#2 G#2 D#3 D2 D2 A2 G2 G1 G2 G2 G2 D3'

In [92]:
outputs[0]['generated'][0]

'2 D#2 G2 G#2 D#2 A#2 C3 D#2 D#3 D#2 G2 G2 C2 C2 G2 F2 F1 D#2 D2 C2 B1 A#2 A#1 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D#2 D

In [93]:
prompts = []
gens = []
for i in range(len(outputs)):
  for prompt, gen in zip(outputs[i]['prompt'], outputs[i]['generated']):
    prompts.append(prompt)
    gens.append(gen)

In [94]:
df = pd.DataFrame(list(zip(prompts, gens)), columns=['prompt', 'generation'])
df.to_csv('/content/distilgpt2-test_output.csv')
df.head()

,prompt,generation
0,A#1 G#1 A1 A0 A1 D2 F#2 G2 D3 G2 A#1 A#0 A#1 D...,2 D#2 G2 G#2 D#2 A#2 C3 D#2 D#3 D#2 G2 G2 C2 C...
1,F4 C4 A3 D3 F4 F4 F4 C4 A3 F3 A4 D4 C4 F3 F3 C...,5 A#4 F4 D4 G3 D5 F5 D5 A#4 F4 A#3 D5 C5 A#4 C...
2,C#3 C2 F#2 F#2 D2 F#2 F#2 C2 F#2 F#2 D2 F#2 F#...,D2 F#2 F#2 C2 F#2 F#2 D2 C#3 C2 F#2 C2 F#2 F#2...
3,G2 G4 B3 D4 D3 G3 G4 D3 G3 G2 D4 B3 G2 G4 B3 D...,G3 C4 C3 G4 E4 G2 G4 B3 D4 D3 G3 G4 D3 G3 G2 ...
4,C4 E4 A4 G4 D4 B3 A4 E4 C4 A2 B3 D4 G4 C4 E4 A...,A4 G4 D4 B3 C4 E4 A4 F2 B3 E4 G3 E4 C4 A4 G4 ...
